In [ ]:
# importations des librairies

from __future__ import print_function
from ipywidgets import interact, interactive, fixed,interact_manual
import ipywidgets as widgets

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from statsmodels.tsa.seasonal import seasonal_decompose
import seaborn as sns
import numpy as np

from scipy.stats import pearsonr
from scipy.stats import bartlett
from scipy.stats import shapiro
from scipy.stats import chi2_contingency
from scipy.stats import kendalltau, spearmanr

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_regression, make_circles

import statsmodels.stats.multicomp as multi
import statsmodels.api as sm
from statsmodels.formula.api import ols

sns.plotting_context("talk")
sns.set_palette("Dark2")


# Vérification des tables

In [ ]:
# Lecture des fichiers
produit = pd.read_csv("./data/products.csv")
client = pd.read_csv("./data/customers.csv")
vente = pd.read_csv("./data/transactions.csv")

# Produit

In [ ]:
# Vérification du la table produit
print(produit.head(5))

In [ ]:
# Vérification des variables de la table produit
print(produit.shape)
print(produit.describe(include='all'))

'id_prod' est bel et bien une clé primaire, car ils sont tous uniques.

Cependant, on peut remarquer que certains prix sont négatifs

In [ ]:
# On vérifie quels sont les prix négatifs
print(produit.sort_values(by='price'))

On voit que le prix négatif correspond au produit 'T_0'

In [ ]:
# On retire les prix négatifs
produit = produit[produit['id_prod'] != 'T_0']
print(produit.shape)
print(produit.describe(include='all'))

In [ ]:
# On vérifie comment est codée la catégorie
print("Les différentes valeurs de la colonne categ sont :",produit.categ.unique())

# Client

In [ ]:
# On regarde la table client
client.sample(5)

In [ ]:
# Vérification des variables de la table client
print(client.shape)
print(client.describe(include='all'))

'client_id' est aussi une clé primaire pour les mêmes conditions

In [ ]:
# Vérification de comment est codé le sexe
print("Les différentes valeurs de la colonne sex sont :",client.sex.unique())

# Vente

In [ ]:
# Vérification de la table vente
print(vente.head(5))

On retire les lignes de test

In [ ]:
# Suppression des lignes établies comme test précédement
vente = vente.drop(vente[vente.id_prod == 'T_0'].index)
vente['date'] = pd.to_datetime(vente['date'], format='%Y-%m-%d')

In [ ]:
# Vérification des variables de la table vente
print(vente.shape)
print(vente.describe(include='all',datetime_is_numeric=True))

# Fusion des Tables

In [ ]:
# On fusionne les 3 tables
tableTotale = pd.merge(vente, produit, on='id_prod', how='left')
tableTotale = pd.merge(tableTotale, client, on='client_id', how='left')

print(f' vente shape is :{vente.shape}')
print(f' produit shape is :{produit.shape}')
print(f' client shape is :{client.shape}')
print(f' tableTotale shape is :{tableTotale.shape}')

In [ ]:
# Vérification des variables de la table obtenue
print(tableTotale.shape)
print(tableTotale.describe(include='all', datetime_is_numeric=True))

On remarque qu'il y a des prix et des catégories qui ne sont pas renseignées

In [ ]:
# Identification des prix non renseignés
print(tableTotale[tableTotale['price'].isna()]['id_prod'].unique())

In [ ]:
# Vérification de la table produit pour savoir d'où vient l'erreur
print(produit[produit['id_prod'] == '0_2245'].shape)

In [ ]:
# Vérification de la table vente pour savoir d'où vient l'erreur
print(vente[vente['id_prod'] == '0_2245'].shape)

Ce produit n'apparait pas dans la table produit ce qui explique qui ne possède ni prix ni catégorie, pourtant il a été vendu 221 fois. Sur les 658664 ventes ce nombre est négligeable. D'autant plus si on remarque que le premier nombre de l'id produit correspond à la catégorie et que la catégorie 0 est la catégorie avec les prix les moins chères en moyenne.

In [ ]:
# Suppression des lignes contenant ce produit
tableTotale = tableTotale[tableTotale['id_prod'] != '0_2245']

# Calcul du chiffre d'affaires

In [ ]:
# Calcul de la moyenne mobile du chiffre d'affaires 
chiffreAffaire = tableTotale[['price', 'date', 'id_prod', 'categ']].sort_values(by='date')
plt.figure(figsize=(10, 8))

chiffreAffaireJournalier = tableTotale[['price', 'date']].resample('D', on='date').sum()

chiffreAffaireJournalier['jour'] = chiffreAffaireJournalier['price'].rolling(1).mean().shift(0)
chiffreAffaireJournalier['semaine'] = chiffreAffaireJournalier['price'].rolling(7).mean().shift(-3)
chiffreAffaireJournalier['mois'] = chiffreAffaireJournalier['price'].rolling(31).mean().shift(-15)
chiffreAffaireJournalier['semestre'] = chiffreAffaireJournalier['price'].rolling(183).mean().shift(-91)

"""
sns.lineplot(x="date",y="price",
             label="CA Journalier", data=chiffreAffaireJournalier,
             errorbar=None)
"""
# Time series plot with Seaborn lineplot() with label
sns.lineplot(x="date", y="semestre",
             label="CA Semestrielle",
             data=chiffreAffaireJournalier,
             errorbar=None)

# 7-day rolling average Time series plot with Seaborn lineplot() with label
sns.lineplot(x="date", y="semaine",
             label="CA Hebdomadaire",
             data=chiffreAffaireJournalier,
             errorbar=None)

# Time series plot with Seaborn lineplot() with label
sns.lineplot(x="date", y="mois",
             label="CA Mensuelle",
             data=chiffreAffaireJournalier,
             errorbar=None)

In [ ]:
chiffreAffaireJournalier

# Nombre de Ventes par catégorie

In [ ]:
# Affichage du nombre de ventes par catégorie
plt.figure(figsize=(10,8))
sns.histplot(data=tableTotale, x='date', hue='categ',palette="dark:c_r")



Comme on peut le voir, il manque certaines données sur le mois d'octobre. On va donc complétement effacer les données du mois d'octobre 2021.

In [ ]:
# Suppression des données du mois d'octobre
tailleAv = len(tableTotale.index)
tableTotale = tableTotale.loc[tableTotale['date'].dt.strftime('%Y%m') != pd.Timestamp(year=2021, month=10, day=1).strftime('%Y%m')]
tailleAp = len(tableTotale.index)

print("on a supprimé",tailleAv-tailleAp,"lignes, soit",round(((tailleAv-tailleAp)/tailleAv)*100,2),"% du dataframe original.")

# Calcul du nouveau chiffre d'affaires

In [ ]:
# Caclcul de la nouvelle moyenne mobile du chiffre d'affaires
chiffreAffaire = tableTotale[['price', 'date', 'id_prod', 'categ']].sort_values(by='date')
plt.figure(figsize=(10, 8))

chiffreAffaireJournalier = tableTotale[['price', 'date']].resample('D', on='date').sum()

chiffreAffaireJournalier['jour'] = chiffreAffaireJournalier['price'].rolling(1).mean().shift(0)
chiffreAffaireJournalier['semaine'] = chiffreAffaireJournalier['price'].rolling(7).mean().shift(-3)
chiffreAffaireJournalier['mois'] = chiffreAffaireJournalier['price'].rolling(31).mean().shift(-15)
chiffreAffaireJournalier['semestre'] = chiffreAffaireJournalier['price'].rolling(183).mean().shift(-91)

"""
sns.lineplot(x="date",y="price",
             label="CA Journalier", data=chiffreAffaireJournalier,
             errorbar=None)
"""
# Time series plot with Seaborn lineplot() with label
sns.lineplot(x="date", y="semestre",
             label="CA Semestrielle",
             data=chiffreAffaireJournalier,
             errorbar=None)

# 7-day rolling average Time series plot with Seaborn lineplot() with label
sns.lineplot(x="date", y="semaine",
             label="CA Hebdomadaire",
             data=chiffreAffaireJournalier,
             errorbar=None)

# Time series plot with Seaborn lineplot() with label
sns.lineplot(x="date", y="mois",
             label="CA Mensuelle",
             data=chiffreAffaireJournalier,
             errorbar=None)

# Nouvelles ventes par catégorie

In [ ]:
# Affichage du nouveau nombre de ventes par catégorie
plt.figure(figsize=(10,8))
sns.histplot(data=tableTotale, x='date', hue='categ',palette="dark:c_r")

# Ajout de nouvelles colonnes

Création d'une tranche d'âge

In [ ]:
# Création du fonction permetant d'arrondir un chiffre à la dizaine inférieur
def arrondiNInf(x, n:int):
    x = int(x)
    while x%n !=0:
        x=x-1
    return x

In [ ]:
# Calcul de l'âge en fonction de la date d'achat
tableTotale['age'] = pd.DatetimeIndex(tableTotale['date']).year - tableTotale['birth']

In [ ]:
# Création de la tranche d'âge
inter = 10
tableTotale['trancheAge'] = tableTotale['age'].apply(arrondiNInf,n=inter).apply(str)\
                            +'-'\
                            +(tableTotale['age'].apply(arrondiNInf,n=inter)+inter).apply(str)


In [ ]:
# Vérification de la table
print(tableTotale.shape)
print(tableTotale.describe(include='all', datetime_is_numeric=True))

On ajoute le prix du panier ainsi que le panier moyen par client.

In [ ]:
# Ajout du montant moyen du panier par client
tableTotale = tableTotale.merge(
    tableTotale.groupby('session_id')['price'].sum().reset_index(),
    how='left', on='session_id').rename(columns={'client_id_x': 'client_id','price_x': 'price','price_y': 'montantPanier'})

tableTotale.sort_values(by='session_id',ascending=False)

tableTotale = tableTotale.merge(
    tableTotale.groupby('client_id')['montantPanier'].mean().reset_index(),
    how='left', on='client_id').rename(columns={'montantPanier_x': 'montantPanier','montantPanier_y': 'montantPanierMoyen'})

tableTotale.sort_values(by='session_id',ascending=False)

tableTotale['montantPanierMoyen'] = tableTotale['montantPanierMoyen'].round(decimals=2)

On ajoute la taille du panier ainsi que la taille moyenne du panier par client.

In [ ]:
# Ajout de la taille moyenne du panier par client
tableTotale = tableTotale.merge(
    tableTotale.groupby('session_id')['client_id'].count().reset_index(),
    how='left', on='session_id').rename(columns={'client_id_x': 'client_id','client_id_y': 'taillePanier'})


tableTotale = tableTotale.merge(
    tableTotale.groupby('client_id')['taillePanier'].mean().reset_index(),
    how='left', on='client_id').rename(columns={'taillePanier_x': 'taillePanier','taillePanier_y': 'taillePanierMoyen'})

tableTotale.sort_values(by='session_id',ascending=False)

tableTotale['taillePanierMoyen'] = tableTotale['taillePanierMoyen'].round(decimals=2)

On ajoute le chiffre d'affaires par client

In [ ]:
# Ajout du chiffre d'affaires par client
tableTotale = tableTotale.merge(
    tableTotale.groupby('client_id')['price'].sum().reset_index(),
    how='left', on='client_id').rename(columns={'price_x': 'price','price_y': 'chiffreAffaireClient'})

tableTotale.sort_values(by='client_id')


Ajout de la fréquence d'achat (nombre d'achats moyen par mois)

In [ ]:
# Ajout de la fréquence d'achat par mois
max,min = tableTotale['date'].max(),tableTotale['date'].min()

nbMois = (max - min)/np.timedelta64(1, 'M')
nbMois = round(nbMois)

print("Les données s'étalent sur", nbMois,"mois,",nbMois-1, "si on ne prend pas en compte le mois d'octobre")

On ajoute la fréquence d'achat en ignorant le mois d'octobre

In [ ]:
# Ajout du nombre d'achats total
tableTotale = tableTotale.merge(
    tableTotale.groupby('client_id').count()['date'].reset_index().rename(columns={'date': 'nombre d\'achats'}),
    how='left', on='client_id')

tableTotale = tableTotale.merge(
    tableTotale.groupby('client_id')['date'].agg(['min']).reset_index().rename(columns={'min': 'date_inscription'}),
    how='left', on='client_id')

tableTotale['date_inscription'] = np.where(tableTotale['date_inscription']<'2021-10-15',
                                           tableTotale['date_inscription'] +  pd.DateOffset(months=1),
                                           tableTotale['date_inscription'])

tableTotale['date_inscription'] = pd.to_datetime(tableTotale['date_inscription'])
tableTotale['date'] = pd.to_datetime(tableTotale['date'])




In [ ]:
# Ajout de la fréquence d'achat par mois
dateFinale = tableTotale['date'].max()

tableTotale['ventes_mensuelles'] = round(tableTotale['nombre d\'achats'] / ((dateFinale - tableTotale['date_inscription']).dt.days/30),2)

print(tableTotale.describe(datetime_is_numeric=True))
# print(tableTotale.sort_values(by='price'))

On supprime les colonnes : taille du panier, montant du panier et nombre d'achats devenues inutiles

In [ ]:
# Suppression des colonnes : ["nombre d\'achats", "montantPanier", "taillePanier"]
tableTotale = tableTotale.drop(["nombre d\'achats", "montantPanier", "taillePanier"], axis=1)


(Importance des plus grosses fréquences d'achats)

In [ ]:
# Analyse des fréquence d'achats
f, (ax_box1, ax_box) = plt.subplots(nrows=1, ncols=2, sharex=False,  figsize=(20,10))
sns.boxplot(data=tableTotale['ventes_mensuelles'], showfliers=False, ax=ax_box1)
sns.boxplot(data=tableTotale['ventes_mensuelles'], showfliers=True, ax=ax_box)

On remarque que 4 clients ont une fréquence d'achat anormalement élevée

In [ ]:
# Vérification de quels clients ont les fréquences les plus élevées
clientAnormal = tableTotale[tableTotale['ventes_mensuelles']>17.5]['client_id'].unique()
print(clientAnormal)

Afin d'obtenir une étude plus cohérente, on décide de séparer ces clients des autres

In [ ]:
# Séparation des clients en deux catégories
tableTotaleNormal = tableTotale[~tableTotale['client_id'].isin(clientAnormal)]
tableTotaleAnormal = tableTotale[tableTotale['client_id'].isin(clientAnormal)]

tableTotale['anormal'] = 'Client \'Normal\''
tableTotale.loc[tableTotale['client_id'].isin(clientAnormal),'anormal'] = 'Client \'Anormal\''

In [ ]:
# Création d'une fonction qui permet de détérminer si une colonne est une clé primaire
def cleprim(df,id):

    l=len(df)-len(df[id].drop_duplicates())

    if l==0:
        print("C'est une clé primaire")
    else:
        print("Ce n'est pas une clé primaire, il y a",l,"doublons")

    return

cleprim(tableTotale,['session_id','client_id'])
tableTotale

# Chiffre d'affaires par produit

In [ ]:

# Calcul du chiffre d'affaire par produit en fonction du type de client
chiffreAffaireProduit = tableTotale.copy(deep=True)
chiffreAffaireProduit.set_index('date')
chiffreAffaireProduit.index = pd.to_datetime(chiffreAffaireProduit.index)
chiffreAffaireProduit = chiffreAffaireProduit.groupby(['id_prod','anormal']).sum().reset_index().sort_values(by='price').rename(columns={"price": "Chiffre d'affaires"})

chiffreAffaireProduit['SumAll'] = chiffreAffaireProduit["Chiffre d'affaires"].groupby(chiffreAffaireProduit['id_prod']).transform('sum')
print(chiffreAffaireProduit.sort_values(by='id_prod'))

chiffreAffaireProduit = chiffreAffaireProduit.sort_values(by='SumAll')
sns.barplot(x="Chiffre d'affaires", y="id_prod", data=chiffreAffaireProduit.head(10), hue='anormal')
plt.legend(title='Type de client')
plt.show()

chiffreAffaireProduit = chiffreAffaireProduit.sort_values(by='SumAll',ascending=False)
sns.barplot(x="Chiffre d'affaires", y="id_prod", data=chiffreAffaireProduit.head(20), hue='anormal')
plt.legend(title='Type de client')


# Chiffre d'affaire par catégorie

In [ ]:
# Calcul du chiffre d'affaire par catégorie
plt.figure(figsize=(15,8))
chiffreAffaireCategorie = tableTotale.groupby(by=[pd.Grouper(key='date', freq='M'), 'categ'])['price'].sum().unstack(fill_value=0).reset_index()
chiffreAffaireCategorie['date'] = pd.to_datetime(chiffreAffaireCategorie['date']).dt.date
chiffreAffaireCategorie[1.0] = chiffreAffaireCategorie[1.0] + chiffreAffaireCategorie[0.0]
chiffreAffaireCategorie[2.0] = chiffreAffaireCategorie[1.0] + chiffreAffaireCategorie[2.0]

sns.barplot(x = 'date', y = 2.0, data = chiffreAffaireCategorie, color = 'limegreen')
sns.barplot(x = 'date', y = 1.0, data = chiffreAffaireCategorie, color = 'coral')
sns.barplot(x = 'date', y = 0.0, data = chiffreAffaireCategorie, color = 'slateblue')

plt.ylabel('Chiffre d\'affaire')
plt.xticks(rotation=45)


chiffreAffaireCategorie = tableTotale.copy(deep=True)

plt.figure(figsize=(15,8))
chiffreAffaireCategorie = chiffreAffaireCategorie.groupby(by=[pd.Grouper(key='date', freq='M'), 'categ'])['price'].sum().reset_index()#.unstack(fill_value=0)##
chiffreAffaireCategorie['date'] = pd.to_datetime(chiffreAffaireCategorie['date']).dt.date

sns.barplot(data=chiffreAffaireCategorie,x='date',y='price',hue='categ')
plt.xticks(rotation=45)
plt.ylabel('Chiffre d\'affaires')
plt.show()

"""
sns.lineplot(data=chiffreAffaireCategorie,x='date',y='price',hue='categ')
#chiffreAffaireCategorie.plot.bar(stacked=True, x='date')
plt.show()
print(chiffreAffaireCategorie)
"""



In [ ]:
# !!!! A SUPPRIMER | Graphique chiffre d'affaires par date, catégorie et type de client
chiffreAffaireCategorie = tableTotale.groupby(by=[pd.Grouper(key='date', freq='M'), 'categ', 'anormal'])['price'].sum().unstack(fill_value=0).reset_index()

chiffreAffaireCategorie['date'] = pd.to_datetime(chiffreAffaireCategorie['date']).dt.strftime('%m/%Y')

chiffreAffaireCategorie = pd.melt(chiffreAffaireCategorie, id_vars=["date",'categ'], var_name="Type_client", value_name="Chiffre d\'affaires total")

chiffreAffaireCategorie = chiffreAffaireCategorie.sort_values(by='date')

hue = chiffreAffaireCategorie[['categ', 'Type_client']].apply(lambda row: f"{row.categ}, {row.Type_client}", axis=1)
hue.name = 'Catégorie, Type_client'

f,(ax1,ax2) = plt.subplots(2, figsize=(50,10))

ax2.legend_ = None

hue_order=["0.0, Client 'Anormal'","0.0, Client 'Normal'","1.0, Client 'Anormal'","1.0, Client 'Normal'","2.0, Client 'Anormal'","2.0, Client 'Normal'"]
sns.barplot(x='date', y="Chiffre d\'affaires total", hue=hue,hue_order=hue_order, data=chiffreAffaireCategorie.iloc[:72,:], palette='Paired',ax=ax1)

sns.barplot(x='date', y="Chiffre d\'affaires total", hue=hue,hue_order=hue_order, data=chiffreAffaireCategorie.iloc[72:,:], palette='Paired',ax=ax2)



# Répartition des clients

Courbe de Lorenz

In [ ]:
# Fonction courbe de Lorenz
def lorenz(variable, title, ax):
    X = variable.values
    X = np.sort(X)

    # Indice de Gini
    def gini(array):
        sorted_array = array.copy()
        sorted_array.sort()
        n = array.size
        coef_ = 2. / n
        const_ = (n + 1.) / n
        weighted_sum = sum([(i + 1) * yi for i, yi in enumerate(sorted_array)])
        return coef_ * weighted_sum / (sorted_array.sum()) - const_

    print('Incide de Gini :', gini(X))

    # Courbe de Lorenz
    X_lorenz = X.cumsum() / X.sum()
    X_lorenz = np.insert(X_lorenz, 0, 0)
    # X_lorenz[0], X_lorenz[-1]
    y = np.arange(X_lorenz.size) / (X_lorenz.size - 1)
    lorenz = pd.DataFrame()
    lorenz['X'] = pd.Series(X_lorenz)
    lorenz['Y'] = pd.Series(y)
    sns.scatterplot(data=lorenz, x='Y', y='X', marker='x', ax=ax)

    # Diagonale
    a = np.arange(0, 1, .01)
    x = a
    y = a

    # Graphique
    sns.lineplot(x=x, y=y, ax=ax)
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    ax.set(xlabel=title)


In [ ]:
# Affichage des courbes de Lorenz
chiffreAffaireClient = tableTotale.groupby('client_id')['price'].sum().rename('chiffre d\'affaire').sort_values()
chiffreAffaireClientNormaux = tableTotaleNormal.groupby('client_id')['price'].sum().rename('chiffre d\'affaire').sort_values()

fig, (ax1,ax2) = plt.subplots(2, figsize=(10,10))

#lorenz(chiffreAffaireClientNormaux, 'Répartition du chiffre d\'affaires les clients \'normaux\'', ax1)
#lorenz(chiffreAffaireClient, 'Répartition du chiffre d\'affaires entre tous les clients', ax2)

# Corrélation entre les caractéristiques

In [ ]:
# Calcul des coefficients de Pearson entre les caractéristiques
print(tableTotale.columns)
"""sns.heatmap(tableTotale.replace({"m": 0,"f": 1,"Client 'Normal'": 0,"Client 'Anormal'": 1}).corr(method='pearson'), annot = True, fmt='.2g',cmap= 'YlGnBu')"""

mask = np.triu(np.ones_like(tableTotaleNormal.replace({"m": 0,"f": 1}).corr()))

sns.heatmap(tableTotaleNormal.replace({"m": 0,"f": 1}).corr(method='pearson'),vmin=-1,vmax=1, annot = True, mask=mask, fmt='.2g',cmap= 'coolwarm')

mask = np.triu(np.ones_like(tableTotale.replace({"m": 0,"f": 1,"Client 'Normal'": 0,"Client 'Anormal'": 1}).corr()))

plt.show()
sns.heatmap(tableTotale.replace({"m": 0,"f": 1,"Client 'Normal'": 0,"Client 'Anormal'": 1}).corr(method='pearson'),vmin=-1,vmax=1, annot = True, mask=mask, fmt='.2g',cmap= 'coolwarm')

In [ ]:
# A FINIR | Test statistiques
import statsmodels.api as sm
from statsmodels.formula.api import ols
model = ols('price ~ sex', data=tableTotale).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

Lcategorie = ['price', 'categ', 'sex', 'age', 'nombre d\'achats', 'ventes_mensuelles', 'anormal']
# sns.pairplot(tableTotale.loc[:,Lcategorie].replace({"m": 0,"f": 1,"Client 'Normal'": 0,"Client 'Anormal'": 1}))

In [ ]:
# Éxportation de la matrice des corrélation
corr_mat = tableTotale.replace({"m": 0,"f": 1,"Client 'Normal'": 0,"Client 'Anormal'": 1}).corr(method='pearson')

sorted_mat = corr_mat.unstack().sort_values()

# sorted_mat.to_excel(excel_writer = "test.xlsx")

# Corrélation entre l'âge et les autres caractéristiques

On commence par étudier combien il y a de clients par tranche d'âge.

In [ ]:
# Vérification du nombre d'individus par tranche d'âge
data = tableTotale[tableTotale['anormal']=="Client 'Normal'"].groupby(by='client_id').mean().merge(tableTotale[['sex','client_id','anormal']], how='inner', on='client_id').drop_duplicates()
data['age'] = data['age'].apply(int)
data['trancheAge'] = data['age'].apply(arrondiNInf,n=inter).apply(str)+'-'+(data['age'].apply(arrondiNInf,n=inter)+inter).apply(str)

plt.figure(figsize=(20,10))
ax = sns.histplot(data=data.sort_values(by='age'), x='trancheAge', hue='sex', multiple='dodge')

for container in ax.containers:
    ax.bar_label(container)

On remarque que la catégorie est 90-100 n'est pas représenté dans nos clients. Et qu'il y a autant d'hommes que de femmes dans chaque tranche d'âge.

age	nombre d'achats 	0,019005146
ventes_mensuelles	age		0,019041955
age	sex		0,041151568
age	categ		0,096243004
age	price		0,208040753

Le coefficient de pearson entre l'âge et le nombre d'achats (ou de ventes) est très faible (0.02), ce qui laisse sous-entendre qu'il ni a pas de corrélation entre ces deux caractéristiques. Vérifions cela à l'aide de diagramme en boite en prenant le soin d'exclure les 4 clients anormaux.

In [ ]:
# Affichage de la fréquence d'achats en fonction de l'âge et du sexe
plt.figure(figsize=(10,5))
sns.boxplot(data=tableTotaleNormal.sort_values(by='age'), x='trancheAge', y="nombre d'achats",hue='sex')
plt.show()
plt.figure(figsize=(10,5))
sns.boxplot(data=tableTotaleNormal.sort_values(by='age'), x='trancheAge', y="ventes_mensuelles",hue='sex')

Voyons maintenant la taille moyenne du panier et son montant moyen par âge

In [ ]:
# Affichage de la taille et du montant moyen du panier en fonction de la tranche d'âge
plt.figure(figsize=(10,5))
sns.boxplot(data=tableTotaleNormal.drop_duplicates(subset=['client_id']).sort_values(by='age'), x='trancheAge', y="montantPanierMoyen",hue='sex')

plt.figure(figsize=(10,5))
sns.boxplot(data=tableTotaleNormal.drop_duplicates(subset=['client_id']).sort_values(by='age'), x='trancheAge', y="taillePanierMoyen",hue='sex')

In [ ]:
# Affichage de la taille et du montant moyen du panier en fonction de l'âge
plt.figure(figsize=(20,10))
sns.boxplot(data=tableTotaleNormal.drop_duplicates(subset=['client_id']).sort_values(by='age'), x='age', y="montantPanierMoyen")

plt.figure(figsize=(20,10))
sns.boxplot(data=tableTotaleNormal.drop_duplicates(subset=['client_id']).sort_values(by='age'), x='age', y="taillePanierMoyen")

On peut remarquer que les 30-50 ans sont les clients avec le plus grand nombre d'achats en moyenne.

On va ensuite s'intéresser à la relation entre l'âge, le sexe et la catégorie du livre acheté.

In [ ]:
# Affichage du nombre d'achats dans chaque catégorie en fonction de l'âge et du sexe
plt.figure(figsize=(20,10))
sns.displot(data=tableTotaleNormal.sort_values(by='age'), x='trancheAge', hue='sex',multiple='dodge',col='categ')

Grâce à ces graphiques, on peut se rendre compte que le sexe n'a presque aucune incidence sur le comportement d'achat du client (Cp = 0.04). Au contraire, la catégorie achetée à un lien très faible avec l'âge.

Voyons plus en détail la relation entre l'âge et la catégorie achetée.

In [ ]:
# Affichage du nombre de ventes par catégorie et par âge
plt.figure(figsize=(20,10))
sns.histplot(data=tableTotaleNormal.sort_values(by='age'), x='age', hue='categ',multiple='stack',bins=93-17)

On peut voir plus facilement la répartition des catégories par tranche d'âge grâce à ce graphique, les 10-30 ans achètent environ autant de chaque catégorie, tandis que les 30+ n'achètent presque aucun livre de la deuxième catégorie.

Voyons maintenant la corrélation entre l'âge et le prix du livre acheté moyen.

In [ ]:
# Affichage du prix moyen du livre acheté en fonctionde l'âge (et de la catégorie)
plt.figure(figsize=(20,10))
sns.boxplot(data=tableTotaleNormal.sort_values(by='age'), x='trancheAge', y='price')

plt.figure(figsize=(20,10))
sns.boxplot(data=tableTotaleNormal.sort_values(by='age'), x='trancheAge', y='price', hue='categ')

On peut remarquer que les 10-30 ans achètent des livres aux prix moyens les plus élevés. Mais on se rend compte sur le deuxième graphique que cet ecart est due à la différence du prix moyen entre chaque catégorie.

# Corrélation entre le prix et la catégorie

In [ ]:
# Affichage du prix en fonction de la catégorie
plt.figure(figsize=(20,10))
sns.boxplot(data=tableTotaleNormal.drop_duplicates(subset=['id_prod']).sort_values(by='categ'), x='price', y='categ', orient='h')


On peut remarquer que la catégorie 2 est bien plus chère en moyenne que les deux autres.

# Influence de ces différentes caractéristiques sur le chiffre d'affaires

# Test

In [ ]:
tableTotaleNormal.sort_values(by='session_id')